In [2]:
import pandas as pd

In [80]:
train_data = pd.read_json('raw_data/webis-clickbait-22/train.jsonl',lines=True)


In [81]:
col1=[]
col2=[]

for idx in range(len(train_data)):
    concatlen=0
    line_occuring=start_sent=train_data['spoilerPositions'][idx][0][0][0]

    for i in range(line_occuring):
        concatlen+=len(train_data['targetParagraphs'][idx][i])


    starting_position=concatlen+train_data['spoilerPositions'][idx][0][0][1]

    col1.append({'answer_start': [starting_position], 'text': train_data['spoiler'][idx]})


    l=''
    for line in train_data['targetParagraphs'][idx]:
        l+=line
    
    col2.append(l)



In [82]:
df = pd.DataFrame(list(zip(col1, col2)),
               columns =['answers', 'context'])

In [83]:
df

,answers,context
0,"{'answer_start': [692], 'text': ['how about th...",It’ll be just like old times this weekend for ...
1,"{'answer_start': [0], 'text': ['2070']}",2070 is shaping up to be a great year for Moth...
2,"{'answer_start': [271], 'text': ['intellectual...","Despite common belief, money isn't the key to ..."
3,"{'answer_start': [2280], 'text': ['Purpose con...","It’s common wisdom. Near gospel really, and no..."
4,"{'answer_start': [650], 'text': ['in a rice co...","Boiling rice may seem simple, but there is a v..."
...,...,...
3195,"{'answer_start': [1751], 'text': ['it hasn’t n...","A long time ago in a galaxy far, far away...Wa..."
3196,"{'answer_start': [317], 'text': ['It read, ""Th...","The Kansas City, Kansas Police Department are ..."
3197,"{'answer_start': [805], 'text': ['1. Anti-wrin...","Obama looks decades younger already, but what ..."
3198,"{'answer_start': [405], 'text': ['@beyoncefan6...",What the HELL?!1. Unless you’re somewhere with...


In [84]:
df['postText']=train_data['postText'].apply(lambda x: x[0])

In [86]:
df['id']=train_data['postId']

In [87]:
df

,answers,context,postText,id
0,"{'answer_start': [692], 'text': ['how about th...",It’ll be just like old times this weekend for ...,"Wes Welker Wanted Dinner With Tom Brady, But P...",532quh
1,"{'answer_start': [0], 'text': ['2070']}",2070 is shaping up to be a great year for Moth...,NASA sets date for full recovery of ozone hole,411701128456593408
2,"{'answer_start': [271], 'text': ['intellectual...","Despite common belief, money isn't the key to ...",This is what makes employees happy -- and it's...,380537005123190784
3,"{'answer_start': [2280], 'text': ['Purpose con...","It’s common wisdom. Near gospel really, and no...",Passion is overrated — 7 work habits you need ...,844567852531286016
4,"{'answer_start': [650], 'text': ['in a rice co...","Boiling rice may seem simple, but there is a v...",The perfect way to cook rice so that it's perf...,814186311573766144
...,...,...,...,...
3195,"{'answer_start': [1751], 'text': ['it hasn’t n...","A long time ago in a galaxy far, far away...Wa...",Has Facebook's video explosion completely shak...,591979258442027008
3196,"{'answer_start': [317], 'text': ['It read, ""Th...","The Kansas City, Kansas Police Department are ...",Cop Is Eating At A Chili's When Teen Hands Him...,881531941974661_902284433232745
3197,"{'answer_start': [805], 'text': ['1. Anti-wrin...","Obama looks decades younger already, but what ...",5 popular myths about visible signs of aging t...,837356193576333314
3198,"{'answer_start': [405], 'text': ['@beyoncefan6...",What the HELL?!1. Unless you’re somewhere with...,You need to see this Twitter account that pred...,826912122529521666
